## Tensorflow Intro

텐서플로(TensorFlow)는 머신러닝 프로그램, 특히 딥러닝 프로그램을 쉽게 구현할 수 있도록 다양한 기능을 제공해주는 머신러닝 라이브러리로, 모두가 잘 아는 구글에서 만들었습니다.

텐서플로 자체는 기본적으로 C++로 작성했지만 파이썬, 자바, 고(Go) 등 다양한 언어를 지원합니다. 다만, 파이썬을 최우선으로 지원하여 대다수 편의 기능이 파이썬 라이브러리로만 구현되어 있으니 되도록이면 파이썬으로 개발하는 것이 가장 편리합니다.

또한 윈도우, 맥, 리눅스뿐만 아니라 안드로이드, iOS, 라즈베리 파이 등 다양한 시스템에서 쉽게 사용할 수 있도록 지원하여 매우 다양한 곳에 응용할 수 있습니다.

물론 머신러닝/딥러닝을 위한 라이브러리로 텐서플로가 유일한 것은 아닙니다. 토치(Torch), 카페(Caffe), MXNet, 체이너(Chainer), CNTK 등 많은 라이브러리가 있습니다. 그렇다면 왜 텐서플로를 사용하는 것일까요? 제가 생각하는 답은 커뮤니티입니다. 특히 저 같은 엔지니어에게 있어서 라이브러리를 선택할 때 가장 중요한 기준은 커뮤니티라고 생각합니다. 실무에 적용했을 때 생기는 문제점들을 해결하거나, 라이브러리 자체에 버그가 있을 때 얼마나 빠르게 수정되는가 하는 그런 것들. 바로 그런 요인들이 실무를 하는 엔지니어에게는 가장 중요한 부분이라고 할 수 있을 것입니다.

그런 점에 있어 현존하는 머신러닝 라이브러리 중 커뮤니티가 가장 북적이는 것이 바로 텐서플로입니다. 깃허브의 텐서플로 저장소나 각종 애플리케이션, 클라우드 서비스 등은 물론, 새로운 논문이 나올 때마다 텐서플로로 된 구현체가 가장 먼저 나올 정도로 텐서플로 커뮤니티는 놀라울 만큼 활발하게 움직이고 있습니다.

# import, constant, session

In [1]:
# 텐서플로우 import
import tensorflow as tf

# v1 호환성
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior() 

Instructions for updating:
non-resource variables are not supported in the long term


In [2]:
# tf.constant: 말 그대로 상수
hello = tf.constant('Hello, TensorFlow!')
print(hello)

Tensor("Const:0", shape=(), dtype=string)


In [3]:
a = tf.constant(10)
b = tf.constant(32)
c = tf.add(a, b)  # a + b 로도 쓸 수 있음
print(c)

Tensor("Add:0", shape=(), dtype=int32)


In [4]:
# 위에서 변수와 수식들을 정의했지만, 실행이 정의한 시점에서 텐서플로우 학습이 실행되는 것은 아닙니다.
# 아래처럼 Session 객체와 run 메소드를 사용할 때 계산이 됩니다.

# 따라서 모델을 구성하는 단계, 실행하는 단계를 분리하여 프로그램을 깔끔하게 작성할 수 있습니다.

In [5]:
# 그래프를 실행할 세션 구성
sess = tf.Session()
# sess.run: 설정한 텐서 그래프(변수나 수식 등등)를 실행합니다.
print(sess.run(hello))
print(sess.run([a, b, c]))

# 세션을 닫습니다.
sess.close()

b'Hello, TensorFlow!'
[10, 32, 42]


# placeholder, variable

placeholder: 계산에서 입력값을 받는 변수들  
Variable : 계산하면서 최적화 할 변수들

In [6]:
# tf.placeholder: 계산을 실행할 때 입력값을 받는 변수로 사용합니다.
# None 은 크기가 정해지지 않았음을 의미합니다.
X = tf.placeholder(tf.float32, [None, 3])
print(X)

Tensor("Placeholder:0", shape=(?, 3), dtype=float32)


그런데, 막상 실행해보면, a*b+c의 값이 아니라 다음과 같이 Tensor… 라는 문자열이 출력된다. 실제로 값을 뽑아내려면, 이 정의된 그래프에 a,b,c 값을 넣어서 실행해야 하는데, 세션 (Session)을 생성해야 한다.

In [7]:
# X 플레이스홀더에 넣을 값 입니다.
# 플레이스홀더에서 설정한 것 처럼, 두번째 차원의 요소의 갯수는 3개 입니다.
x_data = [[1, 2, 3], [4, 5, 6]]

In [8]:
# tf.Variable: 그래프를 계산하면서 최적화 할 변수들입니다. 이 값이 바로 신경망을 좌우하는 값들입니다.
# tf.random_normal: 각 변수들의 초기값을 정규분포 랜덤 값으로 초기화합니다.
W = tf.Variable(tf.random_normal([3, 2]))
b = tf.Variable(tf.random_normal([2, 1]))

In [9]:
# 입력값과 변수들을 계산할 수식을 작성합니다.
# tf.matmul 처럼 mat* 로 되어 있는 함수로 행렬 계산을 수행합니다.
# 아직 X에는 어떤 값도 넣지 않았습니다. 
expr = tf.matmul(X, W) + b

In [10]:
# 세션을 열고, 위에서 설정한 Variable 들의 값들을 초기화 하기 위해
# 처음에 tf.global_variables_initializer 를 한 번 실행해야 합니다.
sess = tf.Session()
sess.run(tf.global_variables_initializer())

In [11]:
print("=== x_data ===")
print(x_data)
print("=== W ===")
print(sess.run(W))
print("=== b ===")
print(sess.run(b))

=== x_data ===
[[1, 2, 3], [4, 5, 6]]
=== W ===
[[ 0.2136517  -1.5314494 ]
 [-0.0714187  -0.30341563]
 [-1.2406017   0.32053345]]
=== b ===
[[1.405224 ]
 [0.5947467]]


In [12]:
print("=== expr ===")
# expr 수식에는 X 라는 '입력값'이 필요합니다.
# 변수나 함수처럼 입력값이 필요한 대상을 run 하기 위해서는 feed_dict를 통해 값을 입력합니다.
# 따라서 expr 실행시에는 이 변수에 대한 실제 입력값을 다음처럼 넣어줘야합니다.
# 딥러닝을 진행하다보면 엄청나게 많은 학습을 하는 도중에 값이 수시로 업데이트되는데,
# 학습을 하는 과정에서 최적화되며 유동적인 값이 되는 값을 feed_dict로 넣어줍니다.
print(sess.run(expr, feed_dict={X: x_data})) # expr 안에 X를 입력할 건데, X에 는 x_data가 들어갑니다.

sess.close()

=== expr ===
[[-2.2457666   0.22854364]
 [-6.3513503  -5.1249285 ]]


주의. `tf.placeholder`로 정의한 변수는 반드시 `feed_dict` option (dictionary 타입)으로 지정해야 합니다. 그렇지 않을 경우 Invalid Argument Error

# Linear Regresssion

X 와 Y 의 상관관계를 분석하는 기초적인 선형 회귀 모델을 만들고 실행해봅니다.


In [13]:
x_data = [1, 2, 3]
y_data = [1, 2, 3]

# tf.random_uniform 
# 정규분포 난수를 생성하는 함수.
# 배열의 shape, 최소값, 최대값을 파라미터로 사용합니다.
# 여기서는 [1], -1.0, 1.0을 전달했기 때문에 -1에서 1 사이의 난수를 1개 만듭니다. 
# 결과는 1행 1열의 행렬이 됩니다.
W = tf.Variable(tf.random_uniform([1], -1.0, 1.0))
b = tf.Variable(tf.random_uniform([1], -1.0, 1.0))

In [14]:
# name: 나중에 (텐서보드등으로) 값의 변화를 추적하거나 살펴보기 쉽게 하기 위해 이름을 붙일 수도 있습니다.
X = tf.placeholder(tf.float32, name="X")
Y = tf.placeholder(tf.float32, name="Y")
print(X)  # X_런타임횟수:0  Y_런타임횟수:0로 찍힙니다.. 아직은 신경쓸 필요 없습니다
print(Y)

Tensor("X:0", dtype=float32)
Tensor("Y:0", dtype=float32)


최적화할 변수 : W, b  
입력값으로 사용할 변수 : X, Y

In [15]:
# X 와 Y 의 상관 관계를 분석하기 위한 가설 수식을 작성합니다.
# y = W * x + b
# W 와 X 가 행렬이 아니므로 tf.matmul 이 아니라 기본 곱셈 기호를 사용했습니다.
hypothesis = W * X + b

In [16]:
# 손실(오차) 함수를 작성합니다. 여기서는 Mean Squared Error function으로 작성합니다.
# mean(h - Y)^2 : 예측값과 실제값의 거리를 비용(손실) 함수로 정합니다.
cost = tf.reduce_mean(tf.square(hypothesis - Y)) # 우리의 목표는 이 cost를 최소화하는 것입니다.

In [17]:
# 텐서플로우에 기본적으로 포함되어 있는 함수를 이용해 경사 하강법 최적화를 수행합니다.
optimizer = tf.train.GradientDescentOptimizer(learning_rate=0.1)
# 비용을 최소화 하는 것이 최종 목표. 어떤 수단으로? GradientDescentOptimizer를 minimize하는 방식으로.
train_op = optimizer.minimize(cost)

자, 모델의 정의가 끝났으면 이제 training을 시작해봅시다.

- 우리가 원하는 것은? : cost를 최소화하는 것
- cost를 최소화하기 위해 선택한 건? : GradientDescentOptimizer를 minimize하는 것

In [18]:
# 세션을 생성하고 초기화합니다.
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())

    # 최적화를 100번 수행합니다.
    for step in range(100):
        
        # sess.run 을 통해 train_op 와 cost를 계산합니다. 앞의 예제에서는  하나의 노드만 가져왔지만 복수의 tensor를 받아올 수도 있습니다. (어차피 병렬로 계산됨)
        # 이 때, 가설 수식에 넣어야 할 실제값(Placeholder)을 feed_dict 을 통해 전달합니다.
        _, cost_val = sess.run([train_op, cost], feed_dict={X: x_data, Y: y_data})  # train_op 값에는 관심 없음
              # underscore in python? : https://mingrammer.com/underscore-in-python/
        print(step, "번째 학습\t", "손실(오차)", cost_val, " weight:",sess.run(W), " bias:",sess.run(b))
        
    # 최적화가 완료된 모델에 테스트 값을 넣고 결과가 잘 나오는지 확인해봅니다.
    print("\n=== Test 결과 ===")
    print("X: 5, Y:", sess.run(hypothesis, feed_dict={X: 5}))       # X가 5일 때 Y 값 (W * X + b)
    print("X: 2.5, Y:", sess.run(hypothesis, feed_dict={X: 2.5}))   # X가 2.5일 때 Y 값 (W * X + b)

0 번째 학습	 손실(오차) 11.568146  weight: [0.55935764]  bias: [1.3891613]
1 번째 학습	 손실(오차) 0.38738248  weight: [0.4149593]  bias: [1.287586]
2 번째 학습	 손실(오차) 0.24198903  weight: [0.44596288]  bias: [1.264085]
3 번째 학습	 손실(오차) 0.22897752  weight: [0.45743018]  bias: [1.2328829]
4 번째 학습	 손실(오차) 0.21808271  weight: [0.4706755]  bias: [1.2033342]
5 번째 학습	 손실(오차) 0.20772342  weight: [0.483378]  bias: [1.1743971]
6 번째 학습	 손실(오차) 0.19785638  weight: [0.49579966]  bias: [1.1461664]
7 번째 학습	 손실(오차) 0.18845809  weight: [0.50792]  bias: [1.1186132]
8 번째 학습	 손실(오차) 0.17950617  weight: [0.51974934]  bias: [1.0917226]
9 번째 학습	 손실(오차) 0.17097951  weight: [0.5312943]  bias: [1.0654783]
10 번째 학습	 손실(오차) 0.16285788  weight: [0.5425616]  bias: [1.0398649]
11 번째 학습	 손실(오차) 0.15512198  weight: [0.5535582]  bias: [1.0148673]
12 번째 학습	 손실(오차) 0.14775349  weight: [0.5642903]  bias: [0.9904706]
13 번째 학습	 손실(오차) 0.14073516  weight: [0.57476443]  bias: [0.9666603]
14 번째 학습	 손실(오차) 0.13405015  weight: [0.58498687]  bias: [